In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

In [2]:
!unzip -q ../hw_yolo.zip -d /content/

In [ ]:
!python train.py --img 640 --batch 16 --epochs 150 --data /content/hw_yolo/sours/custom_data.yaml --weights yolov5s.pt --cache

In [ ]:
!python /content/yolov5/detect.py --weights /content/hw_yolo/sours/weight.pt --source /content/hw_yolo/images/train   --data /content/hw_yolo/sours/custom_data.yaml --save-txt

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist         # библиотека базы выборок Mnist
from tensorflow import keras
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Model 1 keras: 3 hidden cnn 
model = keras.Sequential([
        Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2), strides=2),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPooling2D((2, 2), strides=2),
        Conv2D(128, (3,3), padding='same', activation='relu'),
        MaxPooling2D((2, 2), strides=2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
      ])
def train():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    x_train = x_train.astype(np.float32)
    x_train /= 255.0
    x_test = x_test.astype(np.float32)
    x_test /= 255.0

    y_train_cat = keras.utils.to_categorical(y_train, 10)
    y_test_cat = keras.utils.to_categorical(y_test, 10)

    cv2.imwrite('/content/x_train.png',x_train[0])

    x_train = np.expand_dims(x_train, axis=3)
    x_test = np.expand_dims(x_test, axis=3)
  
    learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train_cat, batch_size=32, epochs=7, validation_split=0.2, callbacks=[learning_rate_reduction])
    model.evaluate(x_test, y_test_cat)
    model.save('/content/mnist_letters.h5')
        
train()


In [ ]:
import cv2
import numpy as np
from os import path
import glob
letter_list = []
k = 0
for imageName, textName in zip(sorted(glob.glob('/content/hw_yolo/images/train/*')),
                               sorted(glob.glob('/content/yolov5/runs/detect/exp4/labels/*'))):

    image_file = (imageName)
    fr = open(textName)
    lines = fr.readlines()
    line_nums = len(lines)

    if line_nums > 1: 
      x_mat = []
      contours = np.zeros((line_nums, 4))
      for i in range(line_nums):
          line = lines[i]
          item_mat = line.split(',')
          x_mat.append(item_mat[0:4]) # Получить 4 функции
      fr.close()
      for i in range(line_nums):
        contours[i]= x_mat[i]

      # сортировка по х
      contours = sorted(contours, key = lambda x:x[0],reverse = False)
      print(contours)
      contours_y = contours[0][1]
      contour=[]

      # проверка повторения координат
      i = 0
      while (i < len(contours)-1):
        if contours[i][0] >= contours[i+1][0] - 1:
          del contours[i]
        i+=1
      # сортировка по у
      i = 0
      for cont in contours:
        if cont[1]> contours_y + 5 or cont[1]< contours_y - 5 :
          contour.append(cont)
          del contour[i]
        i += 1
        contour.append(cont) 

      letters = []
      out_size=28
      # преобразование изображения
      img = cv2.imread(image_file)
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      ret, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
      img_erode = cv2.erode(thresh, np.ones((1, 2), np.uint8), iterations=1)
      output = img_erode.copy()

      for contour in contours:
          x = int(contour[0]-8)
          y = int(contour[1]-8)
          w = int(contour[2])
          h = int(contour[3])
          cv2.rectangle(output, (x, y), (x + w, y + h), (70, 0, 0), 1)

          letter_crop = img_erode[y:y + h, x:x + w]
          size_max = max(w, h)
          letter_square = 255 * np.ones(shape=[size_max, size_max], dtype=np.uint8)
          
          if w > h:
              y_pos = size_max//2 - h//2
              letter_square[y_pos:y_pos + h, 0:w] = letter_crop
          elif w < h:
              x_pos = size_max//2 - w//2
              letter_square[0:h, x_pos:x_pos + w] = letter_crop
          else:
              letter_square = letter_crop

          letters.append((x, w ,cv2.resize(letter_square, (out_size, out_size), interpolation=cv2.INTER_AREA)))
      k = k+1
      num = str(k)
      cv2.imwrite('/content/'+ num +'output.png',output)

      letter_list.append(letters)


In [ ]:
# получение заданных маркеров цифр
labels = []
for imageLabel in sorted(glob.glob('/content/hw_yolo/labels/train/*')):
    fr = open(imageLabel)
    lines = fr.readlines()
    line_nums = len(lines)
    if line_nums > 1: 

      x_mat = []
      contours = np.zeros((line_nums, 5))
      for i in range(line_nums):
          line = lines[i]
          item_mat = line.split(' ')
          x_mat.append(item_mat[0:5]) # Получить 4 функции
      fr.close()

      for i in range(line_nums):
        contours[i]= x_mat[i]
      contours = sorted(contours, key = lambda x:x[0],reverse = False)

      contours_y = contours[0][2]
      contour=[]
      i = 0
      for cont in contours:
        if cont[1]> contours_y + 5 or cont[1]< contours_y - 5 :
          contour.append(cont)
          del contour[i]
        i += 1
        contour.append(cont)

      lab_cont = np.zeros((len(contour),1))
      for i in range(len(contour)):
          lab_cont[i] = contour[i][0]
      labels.append(lab_cont)
      

In [108]:
# предсказание 
def mnist_predict_img(model, img):
    img_arr = 1 - img/255
    x = np.expand_dims(img_arr, axis=0)
    res = model.predict(x)
    return str(np.argmax(res))

# проход по массиву изображений
def img_to_str(model, letter_list):
    outp_ex = []
    for j in range(len(letter_list)):
      s_out = ''
      letters = letter_list[j]
      for i in range(len(letters)):
        dn = letters[i+1][0] - letters[i][0] - letters[i][1] if i < len(letters) - 1 else 0
        listb = letters[i][2]
        s_out += mnist_predict_img(model, listb)
        if (dn > letters[i][1]/3):
           s_out += ' '
      outp_ex.append(s_out)            
    return outp_ex

model = keras.models.load_model('/content/mnist_letters.h5')
s_out = img_to_str(model, letter_list)
print(s_out)

['52 72 2 1574 4266', '3072 58 11 25332', '3 12 2120 064 141 23 0', '6 44 11 1411 73 4', '21112622 6 3287', '3476 6666 0 04 7 716525', '50', '1 11 7177', '16 6 6061', '26122021 1 5388 1', '27 2 7', '2512 21 3213', '25 12 2 21 0 5413', '7 1122', '061', '75 747 77 577', '46 04 272 561', '12 07 2444', '25 1196', '61 42 2661', '211156127591 456', '54 157453', '35 22017 1 4 4', '2116 414 66 72 7', '48 12 402 15 145', '55 12 2011 04006 2 1 8622', '63 1 2117 04 531', '244 12 212 1 537 04 1415', '24 876 22 4', '4 183', '36 640', '75 12 2011 5135', '36 12 2041 21515', '50 1 2', '21 16 7261', '26122061 24 16 4112', '7 6642 0241 1224 260527116', '151 55 41', '7416 2 61 1 4613 21 0 5', '6114']
